<a href="https://colab.research.google.com/github/snvssk/data298A/blob/saidev/SolarEnergy/MLModels/SVR.ipynbSVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.linear_model import Ridge

Dataset Import

In [ ]:
dataset = pd.read_csv('./all_combined_city.csv')
dataset = dataset.sample(frac =.4) 

### Input and Output Values

In [ ]:
dataset['GHI'].isnull().values.sum()

0

In [ ]:
target = dataset['GHI']#keeping only target(ghi) data here
target

157655     495
497019     941
22991        0
232567       0
313867       0
          ... 
42696     1002
202619      44
471380      60
409716     152
366367      21
Name: GHI, Length: 210470, dtype: int64

In [ ]:
input= dataset.drop(['GHI'], axis=1)
input

,Year,Month,Day,Hour,Cloud_Type,Dew_Point,Solar_Zenith_Angle,Surface_Albedo,Wind_Speed,Precipitable_Water,Wind_Direction,Relative_Humidity,Temperature,Pressure
157655,2019,12,28,11.5,0,5.1,61.21,0.120,1.5,1.400,347.0,56.42,13.6,1018
497019,2019,5,4,13.5,0,8.8,28.31,0.130,2.3,1.000,278.0,54.68,18.1,997
22991,2017,4,23,23.5,8,9.0,128.94,0.119,2.0,2.235,261.0,100.00,11.0,1010
232567,2019,4,6,3.5,8,9.4,116.06,0.180,1.7,2.500,179.0,84.77,11.9,1022
313867,2018,11,23,21.5,4,14.4,144.49,0.130,2.4,2.900,255.0,94.54,15.3,1020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42696,2018,6,8,12.0,0,9.4,14.55,0.120,3.8,2.000,290.0,41.28,23.3,1021
202619,2017,7,21,5.5,1,11.0,86.01,0.152,3.0,1.055,235.7,97.91,14.0,1010
471380,2017,11,16,10.0,8,14.0,62.14,0.104,1.9,3.459,245.6,97.51,15.0,1000
409716,2019,5,12,18.0,0,9.4,78.19,0.140,2.8,1.500,258.0,69.30,15.0,1018


Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
#Normalize the dataset
sc = StandardScaler()
input_normalized = sc.fit_transform(input)

Split Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input_normalized, target, test_size = 0.3, random_state = 1)


Training SVR on the Training set

In [ ]:
X_train.shape

(147329, 14)

In [ ]:
y_test.shape

(63141,)

Confustion Matrix


In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = regressor.predict(X_test)

In [ ]:
import math

MSE = np.square(np.subtract(y_test, y_pred)).mean() 
 
RMSE = math.sqrt(MSE)



In [ ]:
r2_score(y_test, y_pred)

0.9381363969181051

In [ ]:
MSE

5684.7717588890655

In [ ]:
RMSE

75.39742541286847

Need Gridsearch for Hyper paramater Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Parameters for tuning
parameters = {'kernel': ['rbf', 'poly', 'sigmoid'] ,
               'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2,],
               'C': [1, 10, 100, 1000, 10000]}


In [ ]:
svr_gs = GridSearchCV(SVR(epsilon = 0.01), 
                   scoring = 'neg_mean_squared_error',
                   param_grid = parameters,
                   verbose = 3,
                   n_jobs = None,
                   cv =2)


In [ ]:
svr_gs.fit(X_train, y_train)

Fitting 2 folds for each of 75 candidates, totalling 150 fits
[CV 1/2] END C=1, gamma=0.0001, kernel=rbf;, score=-112051.859 total time=13.9min
[CV 2/2] END C=1, gamma=0.0001, kernel=rbf;, score=-112057.723 total time=13.8min
[CV 1/2] END C=1, gamma=0.0001, kernel=poly;, score=-137401.239 total time=12.1min
[CV 2/2] END C=1, gamma=0.0001, kernel=poly;, score=-137510.306 total time=12.1min
[CV 1/2] END C=1, gamma=0.0001, kernel=sigmoid;, score=-123827.547 total time=13.0min
[CV 2/2] END C=1, gamma=0.0001, kernel=sigmoid;, score=-123901.339 total time=13.0min
[CV 1/2] END .C=1, gamma=0.001, kernel=rbf;, score=-43702.721 total time=13.3min
[CV 2/2] END .C=1, gamma=0.001, kernel=rbf;, score=-43536.293 total time=13.2min
[CV 1/2] END C=1, gamma=0.001, kernel=poly;, score=-137396.198 total time=12.2min
[CV 2/2] END C=1, gamma=0.001, kernel=poly;, score=-137502.815 total time=12.2min
[CV 1/2] END C=1, gamma=0.001, kernel=sigmoid;, score=-63949.589 total time=12.4min
[CV 2/2] END C=1, gamma=0.

In [ ]:
print("Best parameters:", svr_gs.best_params_)
print("Lowest RMSE: ", (svr_gs.best_score_)**(1/2.0))